# OpenAI Agents with Opik Integration Cookbook

This cookbook demonstrates how to integrate OpenAI's Agent framework with Opik for comprehensive observability and monitoring of your AI agents.

## Installation and Setup

In [1]:
# Install required packages
%pip install opik openai python-dotenv --upgrade --quiet

Note: you may need to restart the kernel to use updated packages.


### Configure Opik for your session:

In [2]:
import opik
opik.configure()

OPIK: Opik is already configured. You can check the settings by viewing the config file at /home/mavrick/.opik.config


### Set up your OpenAI API key:

In [3]:
import os
import getpass
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

### Import Required Libraries

In [4]:
import opik
import os
from openai import OpenAI
import getpass
import json
import time
import random
from typing import Dict, List, Any

### Enable OpenAI Tracking with Opik

In [5]:
from opik.integrations.openai import track_openai
from opik import track


# Create tracked OpenAI client
openai_client = track_openai(OpenAI())

print("✅ OpenAI client configured with Opik tracking!")

✅ OpenAI client configured with Opik tracking!


### Define Tool Functions

In [6]:
def web_search(query: str) -> str:
    """Mock web search function - replace with actual search implementation"""
    # This is a mock function - in practice, you'd integrate with a real search API
    mock_results = {
        "AI safety": "Recent developments in AI safety include constitutional AI, RLHF improvements, and new alignment research from major labs.",
        "machine learning": "Latest ML trends include transformer architectures, multimodal models, and efficient training techniques.",
        "climate change": "Current climate research focuses on renewable energy, carbon capture, and climate modeling improvements."
    }
    
    # Simple keyword matching for demo
    for key in mock_results:
        if key.lower() in query.lower():
            return mock_results[key]
    
    return f"Search results for '{query}': General information about the topic with recent developments and key insights."

def summarize_content(content: str, max_points: int = 5) -> str:
    """Mock summarization function"""
    # In practice, you might use another LLM call or summarization service
    sentences = content.split('. ')
    key_points = sentences[:max_points]
    summary = "Key Points Summary:\n"
    for i, point in enumerate(key_points, 1):
        if point.strip():
            summary += f"{i}. {point.strip()}\n"
    return summary

def get_current_time() -> str:
    """Get current time"""
    return f"Current time: {time.strftime('%Y-%m-%d %H:%M:%S')}"

print("✅ Tool functions defined!")

✅ Tool functions defined!


### Create Research Agent Tools Configuration

In [7]:
@track(project_name="openai-agents-cookbook")
def create_research_agent_tools():
    """Define tools available to the research agent."""
    
    tools = [
        {
            "type": "function",
            "function": {
                "name": "web_search",
                "description": "Search the web for current information on any topic",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The search query to find relevant information"
                        }
                    },
                    "required": ["query"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "summarize_content",
                "description": "Summarize lengthy content into key points",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "content": {
                            "type": "string",
                            "description": "The content to summarize"
                        },
                        "max_points": {
                            "type": "integer",
                            "description": "Maximum number of summary points",
                            "default": 5
                        }
                    },
                    "required": ["content"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_current_time",
                "description": "Get the current date and time",
                "parameters": {
                    "type": "object",
                    "properties": {},
                    "required": []
                }
            }
        }
    ]
    
    return tools

tools = create_research_agent_tools()
print(f"✅ Created {len(tools)} tools for the research agent")

OPIK: Started logging traces to the "openai-agents-cookbook" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=0197c48e-baa2-74fe-9e14-ffb5576f05c8&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.


✅ Created 3 tools for the research agent


The prompt and response messages are automatically logged to Opik and can be viewed in the UI.

![Openai Agents Opik Integration Cookbook](https://raw.githubusercontent.com/comet-ml/opik/main/apps/opik-documentation/documentation/fern/img/cookbook/openai_agents_opik_integration_cookbook.png)

### Simple Research Agent Implementation

In [8]:
@track(project_name="openai-agents-cookbook")
def run_simple_agent(user_message: str, model: str = "gpt-4o-mini"):
    """Run a simple research agent conversation"""
    
    messages = [
        {
            "role": "system", 
            "content": """You are a helpful research assistant. Use the available tools to provide comprehensive and accurate answers. 
            
            Guidelines:
            - Always search for current information when asked about recent developments
            - Summarize long content to make it digestible
            - Include timestamps when relevant
            - Be helpful and thorough in your responses"""
        },
        {"role": "user", "content": user_message}
    ]
    
    # Make the API call
    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools,
        tool_choice="auto",
        temperature=0.7
    )
    
    assistant_message = response.choices[0].message
    messages.append(assistant_message)
    
    # Handle tool calls if any
    if assistant_message.tool_calls:
        for tool_call in assistant_message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            print(f"🔧 Calling function: {function_name}")
            print(f"📝 Arguments: {function_args}")
            
            # Execute the function
            if function_name == "web_search":
                function_result = web_search(function_args["query"])
            elif function_name == "summarize_content":
                function_result = summarize_content(
                    function_args["content"], 
                    function_args.get("max_points", 5)
                )
            elif function_name == "get_current_time":
                function_result = get_current_time()
            else:
                function_result = f"Unknown function: {function_name}"
            
            print(f"✅ Function result: {function_result[:100]}...")
            
            # Add the function result to the conversation
            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_result
            })
        
        # Get final response after tool execution
        final_response = openai_client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0.7
        )
        
        return final_response.choices[0].message.content, messages
    
    return assistant_message.content, messages

print("✅ Simple agent function ready!")

✅ Simple agent function ready!


### Test Simple Agent

In [9]:


# Test the simple agent
result, conversation = run_simple_agent(
    "What are the latest developments in AI safety research? Please search for current information and summarize the key points."
)

print("🤖 Agent Response:")
print("=" * 50)
print(result)
print("=" * 50)
print(f"\n📊 Conversation had {len(conversation)} messages")

🔧 Calling function: web_search
📝 Arguments: {'query': 'latest developments in AI safety research 2023'}
✅ Function result: Recent developments in AI safety include constitutional AI, RLHF improvements, and new alignment res...
🤖 Agent Response:
Recent developments in AI safety research as of 2023 include several key advancements:

1. **Constitutional AI**: This approach aims to guide AI behavior using a set of ethical principles or "constitution" rather than relying solely on human-generated instructions. It attempts to create a framework for AI systems to make decisions aligned with human values while allowing for flexibility in interpretation.

2. **Improvements in Reinforcement Learning from Human Feedback (RLHF)**: Researchers are refining RLHF techniques to enhance how AI models learn from human input. This includes better methodologies for integrating human preferences into training processes, which could lead to safer and more aligned AI systems.

3. **New Alignment Research**: 

The prompt and response messages are automatically logged to Opik and can be viewed in the UI.

![Openai Agents Opik Integration Simple Agent ](https://raw.githubusercontent.com/comet-ml/opik/main/apps/opik-documentation/documentation/fern/img/cookbook/openai_agents_opik_integration_cookbook_simple_agent.png)

###  Advanced Multi-Turn Agent

In [10]:
@track(project_name="openai-agents-cookbook")
def run_multi_turn_agent(user_message: str, max_iterations: int = 5, model: str = "gpt-4o-mini"):
    """Run a multi-turn conversation with the research agent"""
    
    messages = [
        {
            "role": "system", 
            "content": """You are an expert research assistant with access to search and summarization tools. 

            Your approach:
            1. Break down complex questions into smaller parts
            2. Search for current information when needed
            3. Synthesize information from multiple sources
            4. Provide well-structured, comprehensive answers
            5. Use tools strategically to enhance your responses
            
            Always be helpful, accurate, and thorough."""
        },
        {"role": "user", "content": user_message}
    ]
    
    iteration = 0
    
    while iteration < max_iterations:
        iteration += 1
        print(f"\n🔄 Iteration {iteration}")
        
        # Make the API call
        response = openai_client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice="auto",
            temperature=0.7
        )
        
        assistant_message = response.choices[0].message
        messages.append(assistant_message)
        
        # Check if the model wants to call a function
        if assistant_message.tool_calls:
            print(f"🔧 Agent wants to use {len(assistant_message.tool_calls)} tool(s)")
            
            for tool_call in assistant_message.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                
                print(f"   • {function_name}: {function_args}")
                
                # Execute the function
                if function_name == "web_search":
                    function_result = web_search(function_args["query"])
                elif function_name == "summarize_content":
                    function_result = summarize_content(
                        function_args["content"], 
                        function_args.get("max_points", 5)
                    )
                elif function_name == "get_current_time":
                    function_result = get_current_time()
                else:
                    function_result = f"Unknown function: {function_name}"
                
                # Add the function result to the conversation
                messages.append({
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_result
                })
        else:
            # No more function calls, return the final response
            print("✅ Agent completed its response")
            return assistant_message.content, messages, iteration
    
    print("⚠️ Max iterations reached")
    return "Max iterations reached", messages, iteration

print("✅ Multi-turn agent function ready!")

✅ Multi-turn agent function ready!


### Test Multi-Turn Agent

In [11]:
# Test the multi-turn agent with a complex query
complex_query = """
I'm writing a report on the current state of artificial intelligence. 
Can you help me gather information on:
1. Recent breakthroughs in AI safety
2. The current time for my report timestamp
3. A summary of the key points for my executive summary

Please be thorough and use your tools as needed.
"""

result, conversation, iterations = run_multi_turn_agent(complex_query, max_iterations=5)

print("🤖 Final Agent Response:")
print("=" * 60)
print(result)
print("=" * 60)
print(f"\n📊 Completed in {iterations} iterations with {len(conversation)} total messages")


🔄 Iteration 1
🔧 Agent wants to use 3 tool(s)
   • web_search: {'query': 'recent breakthroughs in AI safety 2023'}
   • get_current_time: {}
   • web_search: {'query': 'key points artificial intelligence 2023'}

🔄 Iteration 2
🔧 Agent wants to use 2 tool(s)
   • summarize_content: {'content': 'Recent developments in AI safety include constitutional AI, RLHF improvements, and new alignment research from major labs.', 'max_points': 5}
   • summarize_content: {'content': 'General information about the topic with recent developments and key insights.', 'max_points': 5}

🔄 Iteration 3
✅ Agent completed its response
🤖 Final Agent Response:
Here's the information you requested for your report on the current state of artificial intelligence:

### 1. Recent Breakthroughs in AI Safety
Recent developments in AI safety include:
- **Constitutional AI**: A new approach to align AI behavior with human values and ethical principles.
- **Reinforcement Learning from Human Feedback (RLHF) Improvements**: 

The prompt and response messages are automatically logged to Opik and can be viewed in the UI.

![openai agents opik integration multi Agent](https://raw.githubusercontent.com/comet-ml/opik/main/apps/opik-documentation/documentation/fern/img/cookbook/openai_agents_opik_integration_cookbook_multi_agent.png)

### Customer Service Agent Implementation

In [12]:
class CustomerServiceAgent:
    def __init__(self, openai_client):
        self.client = openai_client
        self.tools = self._define_tools()
    
    def _define_tools(self):
        return [
            {
                "type": "function",
                "function": {
                    "name": "lookup_order",
                    "description": "Look up customer order information by order ID",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "order_id": {
                                "type": "string",
                                "description": "The order ID to lookup (format: ORD-XXX)"
                            }
                        },
                        "required": ["order_id"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "process_refund",
                    "description": "Process a refund for a customer order",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "order_id": {
                                "type": "string",
                                "description": "The order ID for refund"
                            },
                            "amount": {
                                "type": "number",
                                "description": "Refund amount in USD"
                            },
                            "reason": {
                                "type": "string",
                                "description": "Reason for refund"
                            }
                        },
                        "required": ["order_id", "amount", "reason"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "escalate_to_human",
                    "description": "Escalate the conversation to a human agent",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "reason": {
                                "type": "string",
                                "description": "Reason for escalation"
                            },
                            "priority": {
                                "type": "string",
                                "enum": ["low", "medium", "high"],
                                "description": "Priority level for escalation"
                            }
                        },
                        "required": ["reason"]
                    }
                }
            }
        ]
    
    def lookup_order(self, order_id: str) -> Dict:
        """Mock order lookup function"""
        time.sleep(0.1)  # Simulate database lookup
        mock_orders = {
            "ORD-123": {
                "status": "shipped",
                "items": ["Wireless Laptop", "Gaming Mouse"],
                "total": 1299.99,
                "tracking": "TRK-456789",
                "shipped_date": "2024-01-15"
            },
            "ORD-456": {
                "status": "processing",
                "items": ["Noise-Cancelling Headphones"],
                "total": 199.99,
                "tracking": None,
                "expected_ship": "2024-01-20"
            },
            "ORD-789": {
                "status": "delivered",
                "items": ["Smartphone Case", "Screen Protector"],
                "total": 29.99,
                "tracking": "TRK-987654",
                "delivered_date": "2024-01-10"
            }
        }
        return mock_orders.get(order_id, {"error": "Order not found"})
    
    def process_refund(self, order_id: str, amount: float, reason: str) -> Dict:
        """Mock refund processing function"""
        time.sleep(0.2)  # Simulate processing time
        return {
            "refund_id": f"REF-{random.randint(1000, 9999)}",
            "status": "processed",
            "amount": amount,
            "order_id": order_id,
            "reason": reason,
            "estimated_days": "3-5 business days",
            "confirmation_email": "sent"
        }
    
    def escalate_to_human(self, reason: str, priority: str = "medium") -> Dict:
        """Mock escalation function"""
        return {
            "ticket_id": f"ESC-{random.randint(10000, 99999)}",
            "status": "escalated",
            "priority": priority,
            "reason": reason,
            "estimated_response": "within 2 hours",
            "agent_assigned": "pending"
        }
    
    @track(project_name="openai-agents-cookbook")
    def handle_customer_request(self, customer_message: str, model: str = "gpt-4o-mini"):
        """Handle a customer service request"""
        
        messages = [
            {
                "role": "system",
                "content": """You are a helpful customer service agent. Your goal is to:

                1. Understand the customer's issue clearly
                2. Use available tools to help resolve their problem
                3. Be empathetic and professional
                4. Escalate to human agents when necessary
                5. Always confirm actions before processing refunds

                Available tools:
                - lookup_order: Find order information
                - process_refund: Process refunds (ask for confirmation first)
                - escalate_to_human: Escalate complex issues

                Always be polite and helpful!"""
            },
            {"role": "user", "content": customer_message}
        ]
        
        max_iterations = 3
        iteration = 0
        
        while iteration < max_iterations:
            iteration += 1
            
            response = self.client.chat.completions.create(
                model=model,
                messages=messages,
                tools=self.tools,
                tool_choice="auto",
                temperature=0.3
            )
            
            assistant_message = response.choices[0].message
            messages.append(assistant_message)
            
            if assistant_message.tool_calls:
                for tool_call in assistant_message.tool_calls:
                    function_name = tool_call.function.name
                    function_args = json.loads(tool_call.function.arguments)
                    
                    print(f"🔧 Using tool: {function_name}")
                    
                    # Execute the appropriate function
                    if function_name == "lookup_order":
                        result = self.lookup_order(function_args["order_id"])
                    elif function_name == "process_refund":
                        result = self.process_refund(
                            function_args["order_id"],
                            function_args["amount"],
                            function_args["reason"]
                        )
                    elif function_name == "escalate_to_human":
                        result = self.escalate_to_human(
                            function_args["reason"],
                            function_args.get("priority", "medium")
                        )
                    else:
                        result = {"error": f"Unknown function: {function_name}"}
                    
                    # Add function result to messages
                    messages.append({
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": json.dumps(result)
                    })
            else:
                return assistant_message.content, messages
        
        return "Maximum iterations reached", messages

# Create customer service agent
cs_agent = CustomerServiceAgent(openai_client)
print("✅ Customer Service Agent created!")

✅ Customer Service Agent created!


### Test Customer Service Agent

In [13]:
# Test scenarios for customer service agent
test_scenarios = [
    "Hi, I'd like to check the status of my order ORD-123",
    "I received order ORD-456 but the headphones don't work. I'd like a refund.",
    "My order ORD-999 never arrived and it's been 2 weeks. This is very frustrating!"
]

for i, scenario in enumerate(test_scenarios, 1):
    print(f"\n{'='*20} Test Scenario {i} {'='*20}")
    print(f"Customer: {scenario}")
    print("-" * 60)
    
    response, conversation = cs_agent.handle_customer_request(scenario)
    
    print(f"Agent: {response}")
    print(f"(Conversation had {len(conversation)} messages)")


==================== Test Scenario 1 ====================
Customer: Hi, I'd like to check the status of my order ORD-123
------------------------------------------------------------
🔧 Using tool: lookup_order
Agent: Your order **ORD-123** has been shipped! Here are the details:

- **Items**: 
  - Wireless Laptop
  - Gaming Mouse
- **Total Amount**: $1299.99
- **Tracking Number**: TRK-456789
- **Shipped Date**: January 15, 2024

If you have any more questions or need further assistance, feel free to ask!
(Conversation had 5 messages)

==================== Test Scenario 2 ====================
Customer: I received order ORD-456 but the headphones don't work. I'd like a refund.
------------------------------------------------------------
Agent: I'm sorry to hear that the headphones you received are not working. I can help you with the refund process. 

Before we proceed, could you please confirm the following details for me?

1. The order ID: ORD-456
2. The amount you would like to be ref

The prompt and response messages are automatically logged to Opik and can be viewed in the UI.

![Openai Agents Opik Integration Handel Customer](https://raw.githubusercontent.com/comet-ml/opik/main/apps/opik-documentation/documentation/fern/img/cookbook/openai_agents_opik_integration_cookbook_handel_customer.png)

### Agent Performance Evaluation

In [14]:
from opik.evaluation.metrics import LevenshteinRatio, Contains
from opik import evaluate

def evaluate_agent_responses():
    """Evaluate agent performance using Opik's evaluation framework"""
    
    # Test dataset for evaluation
    test_cases = [
        {
            "input": "What are recent AI developments?",
            "expected_keywords": ["AI", "research", "development", "recent"]
        },
        {
            "input": "Check order ORD-123 status",
            "expected_keywords": ["order", "status", "ORD-123"]
        },
        {
            "input": "I need help with a refund",
            "expected_keywords": ["refund", "help", "process"]
        }
    ]
    
    # Custom evaluation metric
    def contains_keywords(output: str, expected_keywords: List[str]) -> float:
        """Check if output contains expected keywords"""
        output_lower = output.lower()
        found_keywords = sum(1 for keyword in expected_keywords if keyword.lower() in output_lower)
        return found_keywords / len(expected_keywords)
    
    results = []
    
    for test_case in test_cases:
        # Run agent
        if "order" in test_case["input"].lower():
            response, _ = cs_agent.handle_customer_request(test_case["input"])
        else:
            response, _ = run_simple_agent(test_case["input"])
        
        # Evaluate response
        score = contains_keywords(response, test_case["expected_keywords"])
        
        results.append({
            "input": test_case["input"],
            "output": response,
            "score": score,
            "expected_keywords": test_case["expected_keywords"]
        })
        
        print(f"Input: {test_case['input']}")
        print(f"Score: {score:.2f}")
        print(f"Expected keywords: {test_case['expected_keywords']}")
        print("-" * 40)
    
    avg_score = sum(r["score"] for r in results) / len(results)
    print(f"\n📊 Average Performance Score: {avg_score:.2f}")
    
    return results

evaluation_results = evaluate_agent_responses()

🔧 Calling function: web_search
📝 Arguments: {'query': 'recent AI developments 2023'}
✅ Function result: Search results for 'recent AI developments 2023': General information about the topic with recent de...
Input: What are recent AI developments?
Score: 0.75
Expected keywords: ['AI', 'research', 'development', 'recent']
----------------------------------------
🔧 Using tool: lookup_order
Input: Check order ORD-123 status
Score: 1.00
Expected keywords: ['order', 'status', 'ORD-123']
----------------------------------------
Input: I need help with a refund
Score: 1.00
Expected keywords: ['refund', 'help', 'process']
----------------------------------------

📊 Average Performance Score: 0.92


The prompt and response messages are automatically logged to Opik and can be viewed in the UI.

![Openai Agents Opik Integration Agent Performance](https://raw.githubusercontent.com/comet-ml/opik/main/apps/opik-documentation/documentation/fern/img/cookbook/openai_agents_opik_integration_cookbook_agent_performance.png)

In [15]:
# Instructions for viewing results in Opik
print("🎯 View Your Agent Traces in Opik:")
print("=" * 50)
print("1. Go to https://www.comet.com/")
print("2. Navigate to your Opik project: 'openai-agents-cookbook'")
print("3. Explore the traces to see:")
print("   • Agent conversations and tool usage")
print("   • Performance metrics and timing")
print("   • Token usage and costs")
print("   • Function call patterns")
print("\n💡 Tips for Analysis:")
print("   • Filter traces by project name")
print("   • Compare different agent implementations")
print("   • Monitor tool usage patterns")
print("   • Track performance over time")

🎯 View Your Agent Traces in Opik:
1. Go to https://www.comet.com/
2. Navigate to your Opik project: 'openai-agents-cookbook'
3. Explore the traces to see:
   • Agent conversations and tool usage
   • Performance metrics and timing
   • Token usage and costs
   • Function call patterns

💡 Tips for Analysis:
   • Filter traces by project name
   • Compare different agent implementations
   • Monitor tool usage patterns
   • Track performance over time


## Advanced Features - Custom Metadata

In [16]:
@track(
    project_name="openai-agents-cookbook",
    tags=["advanced", "metadata"],
    metadata={"agent_type": "research", "version": "1.0"}
)
def research_agent_with_metadata(query: str, user_id: str = "demo_user"):
    """Research agent with custom metadata tracking"""
    
    # Run the agent with the query
    response, conversation = run_simple_agent(query)
    
    # The @track decorator will automatically capture the function's input and output
    # Additional metadata is already set in the decorator above
    
    return response

# Test with custom metadata
result = research_agent_with_metadata(
    "What are the latest trends in machine learning?", 
    user_id="researcher_001"
)

print("🎯 Agent Response with Custom Metadata:")
print(result)

🔧 Calling function: web_search
📝 Arguments: {'query': 'latest trends in machine learning 2024'}
✅ Function result: Latest ML trends include transformer architectures, multimodal models, and efficient training techni...
🎯 Agent Response with Custom Metadata:
As of 2024, several key trends are shaping the landscape of machine learning:

1. **Transformer Architectures**: Transformers continue to dominate many NLP and computer vision tasks due to their superior performance and scalability. These models are being adapted for various applications beyond their original design.

2. **Multimodal Models**: There is an increasing focus on models that can process and integrate multiple types of data (e.g., text, images, audio) simultaneously. This trend is driven by the need for more comprehensive understanding and generation of content across different media.

3. **Efficient Training Techniques**: As models grow larger, there is a growing emphasis on developing more efficient training methods. Te

The prompt and response messages are automatically logged to Opik and can be viewed in the UI.

![openai agents opik Integration  Research Agent](https://raw.githubusercontent.com/comet-ml/opik/main/apps/opik-documentation/documentation/fern/img/cookbook/openai_agents_opik_integration_cookbook_research_agent.png)

In [18]:
print("🎉 OpenAI Agents + Opik Integration Complete!")
print("=" * 60)
print("\n✅ What we've covered:")
print("1. Basic agent setup with Opik tracking")
print("2. Simple research agent with tool usage")
print("3. Multi-turn conversation handling")
print("4. Customer service agent implementation")
print("5. Performance evaluation and metrics")
print("6. Custom metadata and tagging")

print("\n📊 Key Benefits of Opik Integration:")
print("• Complete visibility into agent conversations")
print("• Tool usage tracking and analysis")
print("• Performance monitoring and optimization")
print("• Cost tracking and budgeting")
print("• Debug capabilities for complex workflows")

print("\n🚀 Next Steps:")
print("• Explore the Opik dashboard for detailed analytics")
print("• Implement custom evaluation metrics")
print("• Set up alerts for performance monitoring")
print("• Scale to production with proper error handling")
print("• Integrate with your existing agent workflows")

print(f"\n📈 Session Summary:")
print(f"• Project: openai-agents-cookbook")
print(f"• Agents tested: Research Agent, Customer Service Agent")
print(f"• Tools implemented: {len(tools)} research tools + 3 customer service tools")

🎉 OpenAI Agents + Opik Integration Complete!

✅ What we've covered:
1. Basic agent setup with Opik tracking
2. Simple research agent with tool usage
3. Multi-turn conversation handling
4. Customer service agent implementation
5. Performance evaluation and metrics
6. Custom metadata and tagging

📊 Key Benefits of Opik Integration:
• Complete visibility into agent conversations
• Tool usage tracking and analysis
• Performance monitoring and optimization
• Cost tracking and budgeting
• Debug capabilities for complex workflows

🚀 Next Steps:
• Explore the Opik dashboard for detailed analytics
• Implement custom evaluation metrics
• Set up alerts for performance monitoring
• Scale to production with proper error handling
• Integrate with your existing agent workflows

📈 Session Summary:
• Project: openai-agents-cookbook
• Agents tested: Research Agent, Customer Service Agent
• Tools implemented: 3 research tools + 3 customer service tools
